## Individual Recommender system
Running the cell below you can actually put the individual recommender system in practice. Do not add any actual code about the recommenders in this notebook, do that in 'recommender.ipynb' and then import the necessary methods you need. (ask chatgpt how to do so) 

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import random



In [2]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import csv
import keyboard
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor

C:\Users\alexi\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [11]:
ratings_df = pd.read_csv('Data/filtered_ratings.csv')
movies = pd.read_csv('Data/movies.csv')

In [12]:
def get_movie_name(movie_id):
    movie_row = movies[movies['movieId'] == movie_id]

    if not movie_row.empty:
        return movie_row['title'].iloc[0]
    else:
        return "Movie not in the list"

In [13]:
def newUser(movie_ratings):
    new_user_id = 1
    
    # Find the next available user ID
    with open('Data/filtered_ratings.csv', mode='r', newline='') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip the header row
        existing_user_ids = set()
        for row in csv_reader:
            existing_user_ids.add(int(row[0]))
        while new_user_id in existing_user_ids:
            new_user_id += 1

    # Write the new user's ratings to the CSV file, each in a separate row
    with open('Data/filtered_ratings.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        for movie_id, rating in movie_ratings.items():
            writer.writerow([new_user_id, movie_id, rating])
    return new_user_id

In [14]:
def recommendKNN(movie_ratings, new_user_id, ratings_df):
    selected_user = new_user_id 
    selected_user_ratings = ratings_df.loc[ratings_df['userId'] == new_user_id]
    selected_user_ratings['item'] = selected_user_ratings.index
    
    selected_user_ratings = selected_user_ratings.sort_values(by='item', ascending=True)
    selected_movie_ids = set(selected_user_ratings['movieId'])
    
    rated_movies_df = movies.reindex(list(selected_user_ratings['movieId']))
# rated_movies_df = movies_df[movies_df['movieId'].isin(selected_movie_ids)]
    rated_movies_df = rated_movies_df[['title', 'genres']]
    rated_movies_df['item'] = rated_movies_df.index 
    
    diff = set(movies.index) - set(rated_movies_df.index)
    unrated_movies_df = movies.loc[diff]
    unrated_movies_df = unrated_movies_df[['title', 'genres']]
    
    rated_movies_df = rated_movies_df.join(selected_user_ratings.set_index('movieId')['rating'], on='item')
    rated_movies_df['genres'].fillna('', inplace=True)
    
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(rated_movies_df['genres'])
    rated_movies_df['genres'].fillna('', inplace=True)
    
    y = rated_movies_df['rating']
    
    neighbors = 0
    if len(movie_ratings) < 5:
        neighbors = len(movie_ratings)
    else:
        neighbors = 5
    
    neigh = KNeighborsRegressor(n_neighbors=neighbors)
    neigh.fit(X, y)
    
    X_unrated = vectorizer.transform(unrated_movies_df['genres'])
    y_unrated = neigh.predict(X_unrated)
    unrated_movies_df['predicted_ratings_KNN'] = y_unrated
    unrated_movies_df_KNN_ordering = unrated_movies_df.sort_values(by='predicted_ratings_KNN', ascending=False)
    top_10_movies = unrated_movies_df_KNN_ordering.head(10)
    for i, (index, movie) in enumerate(top_10_movies.iterrows(), 1):
        print(f"{i}) Movie: {movie['title']}, Genre: {movie['genres']}")

In [15]:
def printExplanation():
    print("Why Does It Recommend Those Movies:")
    print("Our system recommends movies based on the idea that if you've enjoyed certain movies in the past, you're likely to enjoy movies with similar genres. We analyze the genre data of the movies you've rated and compare it to all the movies in our database. The system identifies movies with similar genre profiles.")
    print("For example, if you loved action-adventure movies, our system is more likely to recommend other action-adventure films.")
    print("It's important to note that the system doesn't take into account factors like actors, directors, or specific plot details. Instead, it solely relies on the genre information and your ratings to make recommendations.")
    print("The more ratings you provide, the better the system becomes at understanding your preferences and offering personalized movie suggestions.")

In [16]:
def deleteRows():
    input_file = "Data/filtered_ratings.csv"

# Read the CSV file into a pandas DataFrame
    df = pd.read_csv(input_file)
    # Find the index of the first row where userId is greater than or equal to 100000
    start_index = df[df['userId'] > 10000].index[0]

    # Keep only the rows up to the start_index
    df = df.iloc[:start_index]

    # Save the updated DataFrame back to the original file, overwriting it
    df.to_csv(input_file, index=False)
    

In [18]:
movie_ratings = {}
while True:
    movie_name = input("Enter a movie name (or 'done' to finish): ")
    
    if movie_name.lower() == 'done' or keyboard.is_pressed('esc'):
        break
    
    matched_movies = process.extractOne(movie_name, movies['title'], scorer=fuzz.token_set_ratio)

    if matched_movies[1] >= 80:  # Similarity threshold
        movie_id = movies[movies['title'] == matched_movies[0]]['movieId'].values[0]
        correct_movie_name = matched_movies[0]
        while True:
            rating = input(f"Enter a rating for '{correct_movie_name}' (1-5): ")
            try:
                rating = float(rating)
                if 1 <= rating <= 5:
                    break
               
                else:
                    print("Rating must be between 1 and 5.")
            except ValueError:
                print("Invalid rating. Please enter a number between 1 and 5.")
        
        movie_ratings[movie_id] = rating
        print(f"Rating for '{correct_movie_name}' added.")
    else:
        print(f"Movie '{movie_name}' not found in the database.")
    
if movie_ratings:
    while True:
        recommendation_type = input("Enter 'genre' for genre-based recommendation or 'knn' for similar user recommendation: ").lower()
        if recommendation_type == 'genre':

            #----------------------------ADD YOUR WORK HERE-----------------------------
            print('ADD HERE YOUR GENRE-BASED RECOMMENDATIONS BASED ON \'movie_ratings\'')
            print('ADD HERE YOUR EXPLANATION ON THE MOVIE RECOMMENDATIONS')
            #----------------------------ADD YOUR WORK HERE-----------------------------

        elif recommendation_type == 'knn':

            #----------------------------ADD YOUR WORK HERE-----------------------------
            user = newUser(movie_ratings)
            ratings_df = pd.read_csv('Data/filtered_ratings.csv')
            print("Your recommendations are:")
            recommendKNN(movie_ratings,user, ratings_df)
            print("\n")
            print("\n")
            printExplanation()
            deleteRows()
#             print('ADD HERE YOUR EXPLANATION ON THE MOVIE RECOMMENDATIONS')
            #----------------------------ADD YOUR WORK HERE-----------------------------

        else:
            print("Invalid choice. Please enter 'genre' or 'user'.")
        break




Enter a movie name (or 'done' to finish): jumanji
Enter a rating for 'Jumanji (1995)' (1-5): 5
Rating for 'Jumanji (1995)' added.
Enter a movie name (or 'done' to finish): shrek
Enter a rating for 'Shrek (2001)' (1-5): 10
Rating must be between 1 and 5.
Enter a rating for 'Shrek (2001)' (1-5): 5
Rating for 'Shrek (2001)' added.
Enter a movie name (or 'done' to finish): shrek 2
Enter a rating for 'Shrek 2 (2004)' (1-5): 5
Rating for 'Shrek 2 (2004)' added.
Enter a movie name (or 'done' to finish): shrek 3
Enter a rating for 'Shrek (2001)' (1-5): 5
Rating for 'Shrek (2001)' added.
Enter a movie name (or 'done' to finish): puss in boots
Enter a rating for 'Puss in Boots (2011)' (1-5): 5
Rating for 'Puss in Boots (2011)' added.
Enter a movie name (or 'done' to finish): blade runne 2048
Movie 'blade runne 2048' not found in the database.
Enter a movie name (or 'done' to finish): blade runner 2048
Enter a rating for 'Blade Runner (1982)' (1-5): 1
Rating for 'Blade Runner (1982)' added.
Enter

C:\Users\alexi\AppData\Local\Temp\ipykernel_20268\4139023092.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_user_ratings['item'] = selected_user_ratings.index
C:\Users\alexi\AppData\Local\Temp\ipykernel_20268\4139023092.py:15: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  unrated_movies_df = movies.loc[diff]


1) Movie: Toy Story (1995), Genre: Adventure|Animation|Children|Comedy|Fantasy
2) Movie: River Why, The (2010), Genre: Drama
3) Movie: From Beyond the Grave (Creatures) (1974), Genre: Horror
4) Movie: Wuthering Heights (2011), Genre: Drama
5) Movie: Wichita (1955), Genre: Action|Romance|Western
6) Movie: Story of Mankind, The (1957), Genre: Drama|Fantasy
7) Movie: Fright (1972), Genre: Crime|Horror|Thriller
8) Movie: Shaolin (Xin shao lin si) (2011), Genre: Action|Drama
9) Movie: Big Hangover, The (1950), Genre: Comedy
10) Movie: Another Happy Day (2011), Genre: Drama




Why Does It Recommend Those Movies:
Our system recommends movies based on the idea that if you've enjoyed certain movies in the past, you're likely to enjoy movies with similar genres. We analyze the genre data of the movies you've rated and compare it to all the movies in our database. The system identifies movies with similar genre profiles.
For example, if you loved action-adventure movies, our system is more likel

In [ ]:
user_ratings_kid = {1: 5.0, 2: 5.0, 364: 4.0, 6377: 4.0}
print(recommendMovie(user_ratings_kid, movies, ratings_df, 5))

In [ ]:
def compute_similarity(genre_rating, contains_genre):
    return 2.75 if not contains_genre else genre_rating

def getMovieGenres(movie_id):
    movie_row = movies[movies['movieId'] == movie_id]

    if not movie_row.empty:
        genres_str = movie_row['genres'].iloc[0]
        
        genres_list = genres_str.split('|')
        
        return genres_list
    else:
        return []
        
def getGenresDict(genres, movie_id):
    movieGenres = getMovieGenres(movie_id)
    genresDict = {}
    for genre, rating in genres.items():  # Iterate through the genres dictionary with keys and values
        if genre in movieGenres:
            genresDict[genre] = True
        else:
            genresDict[genre] = False
    return genresDict


def recommendMovie(user_ratings, movies_df, ratings_df, n):
    movie_ratings_count = ratings_df['movieId'].value_counts().reset_index()
    movie_ratings_count.columns = ['movieId', 'count']
    popular_movies = movies_df.merge(movie_ratings_count, on='movieId', how='inner')
    popular_movies = popular_movies[popular_movies['count'] > 3]
    popular_movies = popular_movies.sort_values(by='count', ascending=False)

    print(len(popular_movies))
    recommended_movies = []
    for movie_id in popular_movies['movieId']:
        if len(recommended_movies) >= n:
            break
        if movie_id not in user_ratings:
            # Use predictRating method to predict the rating
            predicted_rating = predictRating(user_ratings, movie_id)
            if predicted_rating > 3:
                recommended_movies.append(movie_id)

    recommendations_str = ""
    for id in recommended_movies:
        recommendations_str += get_movie_name(id) + "\n"

    return recommendations_str

def getGenreRatings(user_ratings):
    genres = {}
    genre_counts = {}  
    for movie_id, rating in user_ratings.items():
        movie_genres = getMovieGenres(movie_id)
        for current_genre in movie_genres:
            genres[current_genre] = genres.get(current_genre, 0) + rating
            genre_counts[current_genre] = genre_counts.get(current_genre, 0) + 1

    for genre in genres:
        genres[genre] /= genre_counts[genre]  # Divide the total rating by the number of times the genre appears

    return genres

    
getGenreRatings({1: 5.0, 3: 2.5})

def predictRating(user_ratings, movie_to_predict_id):
    weighted_sum = 0.0
    ratings = getGenreRatings(user_ratings)
    contains_genres = getGenresDict(ratings, movie_to_predict_id)

    for movie_id in ratings:
        similarity = compute_similarity(ratings[movie_id], contains_genres[movie_id]) / len(ratings)
        weighted_sum += similarity

    return weighted_sum

ratings = {
    "Action": 5,
    "Drama": 1,
    }

contains_genres = {
    "Drama": True,
    "Action": False,
}

user_ratings_ex = {1: 5.0, 4867: 1.0, 2: 5.0, 364: 4.0, 6377: 4.0}
movie_id_ex = 1

predicted_rating = predictRating(user_ratings_ex, movie_id_ex)
print(predicted_rating)


def getAllGenres(movies_df):
    genres_column = movies_df['genres']
    genre_lists = [genres.split('|') for genres in genres_column]
    flat_genre_list = [genre for sublist in genre_lists for genre in sublist]
    unique_genres = set(flat_genre_list)
    return list(unique_genres)

all_genres = getAllGenres(movies)



In [ ]:
def get_user_ratings(user_id):
    user_ratings_df = ratings_df[ratings_df['userId'] == user_id]
    
    user_ratings = {}
    
    for index, row in user_ratings_df.iterrows():
        movie_id = row['movieId']
        rating = row['rating']
        user_ratings[movie_id] = rating
    
    return user_ratings

max_user_id = 610

### Indivual Recimmender using KNN

In [ ]:
users_ratings = ratings_df.groupby(['userId']).count()

selected = users_ratings['rating'] > 200
selected_users = users_ratings.loc[selected]
random_selected = selected_users.sample() # sample() returns a random row from the dataframe. The returned object is a dataframe with one row. If you pass a number as argument you specify to select more than one row.
select_column_df = random_selected.reset_index()['userId'] # reset_index() create a new index, and the user became a column. Then, we can filter using the column name
selected_user = select_column_df.iloc[0] # iloc select by index, since our dataframe only has one row we read it from the index 0
# selected_user = 19
print("Selected user: " + str(selected_user))

In [ ]:
selected_user_ratings = ratings_df.loc[ratings_df['userId'] == selected_user]
selected_user_ratings['item'] = selected_user_ratings.index 
selected_user_ratings.to_csv('selected_user_ratings.csv', index=False)
selected_user_ratings = selected_user_ratings.sort_values(by='item', ascending=True)
print("Rated movies: " + str(selected_user_ratings.shape[0]))
# display(selected_user_ratings.head(10))

In [ ]:
selected_movie_ids = set(selected_user_ratings['movieId'])
rated_movies_df = movies.reindex(list(selected_user_ratings['movieId']))
# rated_movies_df = movies_df[movies_df['movieId'].isin(selected_movie_ids)]
rated_movies_df = rated_movies_df[['title', 'genres']]
rated_movies_df['item'] = rated_movies_df.index 
rated_movies_df.to_csv('rated_movies_df.csv', index=False)

In [ ]:
diff = set(movies.index) - set(rated_movies_df.index)
unrated_movies_df = movies.loc[diff]
# display(unrated_movies_df.head())
unrated_movies_df = unrated_movies_df[['title', 'genres']]
print("Unrated movies: " + str(unrated_movies_df.shape[0]))
# display(unrated_movies_df.head(10))

In [ ]:
rated_movies_df = rated_movies_df.join(selected_user_ratings.set_index('movieId')['rating'], on='item')
print("Rated movies: " + str(rated_movies_df.shape[0]))
# display(rated_movies_df.head(10))

In [ ]:
rated_movies_df['genres'].fillna('', inplace=True)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(rated_movies_df['genres'])
rated_movies_df['genres'].fillna('', inplace=True)
y = rated_movies_df['rating']
neigh = KNeighborsRegressor(n_neighbors=5)
neigh.fit(X, y)
X_unrated = vectorizer.transform(unrated_movies_df['genres'])
y_unrated = neigh.predict(X_unrated)
unrated_movies_df['predicted_ratings_KNN'] = y_unrated


In [ ]:
user_ratings = movie_ratings  # Use the provided 'movie_ratings' dictionary

# Train a KNN model to predict ratings
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(rated_movies_df['genres'])
y = rated_movies_df['rating']

neigh = KNeighborsRegressor(n_neighbors=5)
neigh.fit(X, y)

#Predict ratings for unrated movies
X_unrated = vectorizer.transform(unrated_movies_df['genres'])
y_unrated = neigh.predict(X_unrated)

#Dictionary with movie IDs and predicted ratings
predicted_ratings = {}
for idx, movie_id in enumerate(unrated_movies_df.index):
    predicted_ratings[movie_id] = y_unrated[idx]

user_and_predicted_ratings = {**user_ratings, **predicted_ratings}

# Sort the movies based on the combined ratings
sorted_movies = sorted(user_and_predicted_ratings.items(), key=lambda x: x[1], reverse=True)

# Print the top 10 recommended movies
print("Top 10 Recommended Movies:")
for i, (movie_id, rating) in enumerate(sorted_movies[:10], 1):
    print(f"{i}. Movie ID: {movie_id}, Rating: {rating}")
for i, (movie_id, rating) in enumerate(sorted_movies[:10], 1):
    print(f"{i}. Movie : {get_movie_name(movie_id)}, Rating: {rating}")

## Group Recommender system
In this case I decided not to ask the users for the ratings because it is counterproductive. In this case, the group ratings will be already defined in the code


In [ ]:
#Some random group users movie ratings
group_movie_ratings = [
    {1: 5, 2: 3, 13: 4, 3: 4.5},
    {190: 2, 9372: 4, 837: 1.5},
    {89: 3.5, 7521: 3, 90: 3.5}
]

In [ ]:
#----------------------------ADD YOUR WORK HERE-----------------------------
print("ADD HERE YOUR MOVIE RECOMMENDATIONS BASED ON \'group_movie_ratings\'")
print('ADD HERE YOUR EXPLANATION ON THE MOVIE RECOMMENDATIONS')
#----------------------------ADD YOUR WORK HERE-----------------------------